TA[MAP]-149.II.NE

Eastern limit starts at 0°0' (12°27'7.1" Greenwich) Western limit starts at 0°7'X" 12 20 7.1 Northern 41°50' Southern 41°45'

° TA[MAP]-149.II.NE_modified.tif.points

mapX,mapY,pixelX,pixelY,enable,dX,dY,residual 12.3353055555555553,41.8333333333333357,772.85005482455812853,-568.71381578947671187,1,3.39804119407631333,0.05842193834280351,3.3985433761420949 12.3353055555555553,41.75,779.65509718216208057,-5318.47694364336075523,1,-3.40700116352763871,8.70991424989551888,9.35255383137737972 12.45197222222222067,41.75,5859.64268279860152688,-5335.86500015021010768,1,-3.33937703571609745,-8.67814225695383357,9.29847256373840203 12.45197222222222067,41.8333333333333357,5852.95496875750905019,-568.86243165705934643,1,3.34833700537637924,-0.09019392923983105,3.34955155900675461

12.335,41.833,772,-568
12.335,41.750,779,-5318
12.451,41.750,5859,-5335
12.451,41.833,5852,-568

In [ ]:
def degrees_to_seconds(deg,min,sec):
    #convert degrees to a whole number to make it easier for addition  
    return (deg*60*60)+(min*60)+sec

def add_degrees(deg1,min1,sec1,deg2,min2,sec2):
    degree1 = degrees_to_seconds(deg1,min1,sec1)
    degree2 = degrees_to_seconds(deg2,min2,sec2)
    return(degree1+degree2)

def add_rome(deg,min,sec):
    #rome_lon_deg = 12
    #rome_lon_min = 27
    #rome_lon_sec = 7.1
    rome_lon_deg = 0
    rome_lon_min = 0
    rome_lon_sec = 0

    return add_degrees(deg,min,sec,rome_lon_deg,rome_lon_min,rome_lon_sec)

def convert_seconds_to_deg_dec(base60):
    sec = base60 % 60
    #print(sec)
    base60_working = base60 - sec
    min_working = (base60_working % (60*60))
    min = min_working/60
    #print(min)
    base60_working = base60_working - min_working
    deg = base60_working/(60*60)
    #print(deg)
    degree_fraction = (min*60+sec)/(60*60)
    return(deg+degree_fraction)

west_of_rome = True
map_lon_origin_dir = "east"
map_lon_origin_deg = 0
map_lon_origin_min = 0
map_lon_origin_sec = 0

#print(add_rome(0,0,0))
#d = add_rome(0,0,0)
#print(convert_seconds_to_deg_dec(d))

#print (degrees_to_seconds(rome_lon_deg,rome_lon_min,rome_lon_sec))

#12.335,41.833,772,-568
#12.335,41.750,779,-5318
#12.451,41.750,5859,-5335
#12.451,41.833,5852,-568

def calculate_points_on_line(lat_lon_static, inc_dec, west_of_meridian, origin_lat_deg, origin_lat_min, origin_lat_sec, origin_lon_deg, origin_lon_min, origin_lon_sec, mins, pixel_origin_x, pixel_origin_y, pixel_end):
    points_lines = ""
    

    if lat_lon_static == "lat":
        static_deg = origin_lat_deg
        static_min = origin_lat_min
        static_sec = origin_lat_sec
        static = convert_seconds_to_deg_dec(degrees_to_seconds(static_deg,static_min,static_sec))
        calc_origin_deg = origin_lon_deg
        calc_origin_min = origin_lon_min
        calc_origin_sec = origin_lon_sec
        calc_origin_seconds = add_rome(calc_origin_deg,calc_origin_min,calc_origin_sec)
        calc_origin_pixel = pixel_origin_x
        
    else:
        static_deg = origin_lon_deg
        static_min = origin_lon_min
        static_sec = origin_lon_sec
        static = convert_seconds_to_deg_dec(add_rome(static_deg,static_min,static_sec))

        calc_origin_deg = origin_lat_deg
        calc_origin_min = origin_lat_min
        calc_origin_sec = origin_lat_sec
        calc_origin_seconds = degrees_to_seconds(calc_origin_deg,calc_origin_min,calc_origin_sec)
        calc_origin_pixel = pixel_origin_y
        
    
    pixel_inc = (pixel_end - calc_origin_pixel)/mins
    min_step = 1
    east_of_meridian = False
    min_seconds = degrees_to_seconds(0,1,0)
    if inc_dec == "dec":
        min_step = -1
    min_west_dec = 1
    if west_of_meridian == True:
        min_west_dec = -1  
        


    for mins in range(0, mins+1):
        point_line = ""
        calc_seconds = calc_origin_seconds + (mins*min_seconds*min_step*min_west_dec)
        calc_deg_dec = convert_seconds_to_deg_dec(calc_seconds)
        calc_pixel = calc_origin_pixel + (mins*pixel_inc)

        if lat_lon_static == "lat":
            # mapX,mapY,pixelX,pixelY,enable,dX,dY,residual 12.3353055555555553,41.8333333333333357,772.85005482455812853,-568.71381578947671187,1,3.39804119407631333,0.05842193834280351,3.3985433761420949
            point_line = str(calc_deg_dec)+","+str(static)+","+str(calc_pixel)+","+str(pixel_origin_y*-1)+",1,0,0\n"
        else:
            point_line = str(static)+","+str(calc_deg_dec)+","+str(pixel_origin_x)+","+str(calc_pixel*-1)+",1,0,0\n"
        points_lines = points_lines + point_line
    return(points_lines) 

point_lines = ""
point_lines = calculate_points_on_line("lat", "inc", True, 41, 50, 0, 0, 0, 0, 7, 5859, 568, 772)
point_lines = point_lines + calculate_points_on_line("lat", "inc", True, 41, 45, 0, 0, 0, 0, 7, 5859, 5318, 772)
point_lines = point_lines + calculate_points_on_line("lon", "dec", False, 41, 50, 0, 0, 0, 0, 5, 5859, 568, 5318)
# Let's not put points not on the border of the map
#point_lines = point_lines + calculate_points_on_line("lon", "dec", False, 41, 50, 0, 0, -7, 0, 5, 772, 568, 5318)
print(point_lines)

with open('ta-map-149-ii-ne.points', 'w') as f:
    f.write("mapX,mapY,pixelX,pixelY,enable,dX,dY,residual\n")  
    f.write(point_lines)
#12.335,41.833,772,-568
#12.335,41.750,779,-5318
#12.451,41.750,5859,-5335
#12.451,41.833,5852,-568

0.0,41.833333333333336,5859.0,-568,1,0,0
-0.01666666666666672,41.833333333333336,5132.285714285715,-568,1,0,0
-0.033333333333333326,41.833333333333336,4405.571428571428,-568,1,0,0
-0.050000000000000044,41.833333333333336,3678.857142857143,-568,1,0,0
-0.06666666666666665,41.833333333333336,2952.1428571428573,-568,1,0,0
-0.08333333333333337,41.833333333333336,2225.4285714285716,-568,1,0,0
-0.09999999999999998,41.833333333333336,1498.7142857142862,-568,1,0,0
-0.1166666666666667,41.833333333333336,772.0,-568,1,0,0
0.0,41.75,5859.0,-5318,1,0,0
-0.01666666666666672,41.75,5132.285714285715,-5318,1,0,0
-0.033333333333333326,41.75,4405.571428571428,-5318,1,0,0
-0.050000000000000044,41.75,3678.857142857143,-5318,1,0,0
-0.06666666666666665,41.75,2952.1428571428573,-5318,1,0,0
-0.08333333333333337,41.75,2225.4285714285716,-5318,1,0,0
-0.09999999999999998,41.75,1498.7142857142862,-5318,1,0,0
-0.1166666666666667,41.75,772.0,-5318,1,0,0
0.0,41.833333333333336,5859,-568.0,1,0,0
0.0,41.81666666666667,5

make a directory for map
lower case
no periods or brackets
ta-map-149-ii-ne

QGIS
Raster | Georeferencer
File | Open Raster
Set Target SRS EPSG:4806 Monte Mario (Rome)
File | Load GCP points

convert to EPSG:8195
reproject 26592
